### Read the data from the input Excel file

In [2]:
import pandas as pd
inputFileName = "DataBlender.xlsx"
paramDF = pd.read_excel(inputFileName, "Param", skiprows=2)
rawMaterialsDF = pd.read_excel(inputFileName, "RawMaterials", skiprows=2)
finishedGoodsDF = pd.read_excel(inputFileName, "FinishedGoods", skiprows=2)

### Set the data into list and dict, easier to manipulate

In [7]:
#Raw materials characteristics
rawMaterials_items = list(rawMaterialsDF['Raw materials'])
costRM = list(rawMaterialsDF['Cost'])
capacityRM = list(rawMaterialsDF['Capacity'])
volumeMinRM = list(rawMaterialsDF['VolumeMin'])
supplierRM = list(rawMaterialsDF['Supplier'])
suppliers_items = list(set(supplierRM))  #supplier list, obtained by removing duplicates

componentsRM = dict()
componentsRM['Component1'] = list(rawMaterialsDF['Component1'])
componentsRM['Component2'] = list(rawMaterialsDF['Component2'])
componentsRM['Component3'] = list(rawMaterialsDF['Component3'])

#Finished goods characteristics
finishedGoods_items = list(finishedGoodsDF['Finished goods'])
demandFG = list(finishedGoodsDF['Demand'])

componentsMinFG = dict()
componentsMinFG['Component1'] = list(finishedGoodsDF['MinComponent1'])
componentsMinFG['Component2'] = list(finishedGoodsDF['MinComponent2'])
componentsMinFG['Component3'] = list(finishedGoodsDF['MinComponent3'])

componentsMaxFG = dict()
componentsMaxFG['Component1'] = list(finishedGoodsDF['MaxComponent1'])
componentsMaxFG['Component2'] = list(finishedGoodsDF['MaxComponent2'])
componentsMaxFG['Component3'] = list(finishedGoodsDF['MaxComponent3'])


### Create your linear program

In [8]:
from pulp import *
prob = LpProblem("BlenderProblem",LpMinimize)

### Add variables

In [9]:
RecipeVar = LpVariable.dicts("Recipe", ((i, j) for i in rawMaterials_items for j in finishedGoods_items), 
                             lowBound=0, cat='Continuous')
RM_VolumeVar = LpVariable.dicts("RMvolume", (i for i in rawMaterials_items), lowBound=0, cat='Continuous')
FG_VolumeVar = LpVariable.dicts("FGvolume", (j for j in finishedGoods_items), lowBound=0, cat='Continuous')

### Add objectives

In [10]:
prob += lpSum([costRM[iRM] * RM_VolumeVar[vRM] for iRM, vRM in enumerate(rawMaterials_items)])

### Add constraints

In [11]:
#Define RM_VolumeVar
for iRM in rawMaterials_items : 
    prob += lpSum([RecipeVar[iRM,iFG] for iFG in finishedGoods_items]) == RM_VolumeVar[iRM], "defVolumeRM[%s]"%(iRM)
    
#Define FG_VolumeVar
for iFG in finishedGoods_items : 
    prob += lpSum([RecipeVar[iRM,iFG] for iRM in rawMaterials_items]) == FG_VolumeVar[iFG], "defVolumeFG[%s]"%(iFG)
    
#Capacity constraint
for iRM, vRM in enumerate(rawMaterials_items) : 
    prob += RM_VolumeVar[vRM] <= capacityRM[iRM], "capacity[%s]"%(iRM)

#Demand constraint
for iFG, vFG in enumerate(finishedGoods_items) : 
    prob += FG_VolumeVar[vFG] >= demandFG[iFG], "demandFG[%s]"%(iFG)
    
#Min & max components in the finished goods
for iComponent in ['Component1', 'Component2', 'Component3']:
    for iFG, vFG in enumerate(finishedGoods_items) : 
        #print(componentsMinFG[iComponent][iFG])
        prob += lpSum([componentsRM[iComponent][iRM] * RecipeVar[vRM,vFG] for iRM, vRM in enumerate(rawMaterials_items)]) >= FG_VolumeVar[vFG] * componentsMinFG[iComponent][iFG]
        prob += lpSum([componentsRM[iComponent][iRM] * RecipeVar[vRM,vFG] for iRM, vRM in enumerate(rawMaterials_items)]) <= FG_VolumeVar[vFG] * componentsMaxFG[iComponent][iFG]

### Solve the problem and store the variable into a dictionnary

In [12]:
#prob.writeLP("blenderProblem.lp", writeSOS=1, mip=1)
prob.solve()
print("Status:", LpStatus[prob.status])
if LpStatus[prob.status] == 1 : #1 for Optimal, 0 for "Not Solved", -1 for "Infeasible", -2 "Unbounded", -3 "Undefined"
    print ("Objective = ", value(prob.objective))
    varsDict = {}
    for v in prob.variables():
        varsDict[v.name] = v.varValue
        if "RMvolume" in v.name:
            print(v.name, "=", v.varValue)



Status: Optimal


import matplotlib.pyplot as plt
import numpy as np

myArray = np.zeros((len(rawMaterials_items), len(finishedGoods_items)), dtype=np.float)
for i in rawMaterials_items:
    for j in finishedGoods_items :
        myArray[i][j] = prob.variables("Recipe_('I1',_'O1')").varValue
plt.imshow(myArray, cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

myArray = np.zeros((len(rawMaterials_items), len(finishedGoods_items)), dtype=np.float)


for iRM, vRM in enumerate(rawMaterials_items):
    for iFG, vFG in enumerate(finishedGoods_items): 
        myArray[iRM][iFG] = varsDict["Recipe_(\'%s\',_\'%s\')" %(vRM, vFG)]
plt.imshow(myArray, cmap='Blues', interpolation='nearest')
plt.show()
print(myArray)